In [20]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
%matplotlib

Using matplotlib backend: Qt5Agg


In [74]:
MF = np.genfromtxt("lin2012xyz2e_1_7sf.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]

(441, 4)


In [95]:
def EdgeOptimalSpectrum(i, delta, L, step=1):
    """
    Returns a spectrum associated with a newtonian optimal color
    """
    sout = np.zeros(int(L/step))
    for j in range(i - int(delta/2), i + int(delta/2), step):
        if( (j >= 0) and (j < L) ):
            sout[j] = 1
    return sout

def DoubleOptimalSpectrum(i, delta, L, step=1):
    """
    Returns a spectrum associated with a true optimal color
    """
    sout = np.zeros(int(L/step))
    for j in range(i - int(delta/2), i + int(delta/2), step):
        if( (j >= 0) and (j < L) ):
            sout[j] = 1
        elif( j > L ):
            sout[j-L] = 1
        elif( j < 0 ):
            sout[L+j] = 1
    return sout

In [96]:
def EdgeOptimal(i, delta, CMF, step=1):
    """
    Optimal windowing function without wrapping
    
    returns XYZ coords of optimal color window defined by i and delta
    
    i: center wavelength-index
    delta: width of the window
    CMF: color matching function
    """
    Norm = np.sum(CMF, 0)
    c0 = 0
    c1 = 0
    c2 = 0
    Spect = EdgeOptimalSpectrum(i, delta, CMF.shape[0], step=1)
    for idx, s in enumerate(Spect):
        if s > 0:
            c0 += s*CMF[idx, 0]
            c1 += s*CMF[idx, 1]
            c2 += s*CMF[idx, 2]
    return (c0/Norm[0], c1/Norm[1], c2/Norm[2])

def DoubleOptimal(i, delta, CMF):
    """
    Optimal windowing function with wrapping
    
    returns XYZ coords of optimal color window defined by i and delta
    
    i: center wavelength-index
    delta: width of the window
    CMF: color matching function
    """
    Norm = np.sum(CMF, 0)
    c0 = 0
    c1 = 0
    c2 = 0
    Spect = DoubleOptimalSpectrum(i, delta, CMF.shape[0], step=1)
    for idx, s in enumerate(Spect):
        if s > 0:
            c0 += s*CMF[idx, 0]
            c1 += s*CMF[idx, 1]
            c2 += s*CMF[idx, 2]
    return (c0/Norm[0], c1/Norm[1], c2/Norm[2])

In [97]:
# plot single-sided Newtonian optimal solid 
C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(0, 441, 10):
    for j in range(0, 441, 20):
        O = EdgeOptimal(i, j, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0], O[1], O[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)


In [99]:
# plot double-sided Schroedinger color solid, with locus of windows 1/2 spectral width

C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(0, 441, 10):
    for j in range(0, 441, 20):
        O = DoubleOptimal(i, j, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0], O[1], O[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)

# plot optimal colors with width = L/2
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = DoubleOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    RGB.append(rgb)
ax.scatter(C0, C1, C2, s=400, c=RGB) 
plt.show()

Observe that we seem to get an optimal color locus when we plot equal width optimal windows in XYZ. Let's verify this

In [98]:
# Plot all optimal colors of constant width on a line
fig = plt.figure()
ax = fig.gca(projection='3d')
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = DoubleOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    ax.scatter(i, 0, s=400, color=rgb) 
plt.show()

To be sure we're doing this right, lets see what happens when we plot with just edge optimals

In [28]:
# Plot only edge optimals of constant width on a line
fig = plt.figure()
ax = fig.gca(projection='3d')
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = EdgeOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    RGB.append(rgb)
    ax.scatter(i, 0, s=400, color=rgb) 
plt.show()

Notice the points close to white and black on the 1/2-width locus. Let's see what the spectrum looks like for those optimal colors

In [104]:
imax = RGB.index(max(RGB))
imin = RGB.index(min(RGB))



plt.plot(DoubleOptimalSpectrum(imax, 220, 441))
plt.plot(DoubleOptimalSpectrum(imin, 220, 441))
plt.legend(["Max", "Min"])


In [36]:
def ColorState(rho):
    """
    TODO
    """